In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Sequential(

            nn.Conv2d(1, 4, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(4),

            nn.Conv2d(4, 4, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(4),

            nn.Conv2d(4, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.01),

            nn.MaxPool2d(2,2),

            nn.Conv2d(8, 12, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(12),

            nn.Conv2d(12, 12, 3, padding= 1),
            nn.ReLU(),
            nn.BatchNorm2d(12),

            nn.Conv2d(12, 16, 3, padding= 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),

            nn.MaxPool2d(2,2),

            nn.Conv2d(16, 20, 3),
            nn.ReLU(),
            nn.BatchNorm2d(20),

            nn.Conv2d(20, 10, 3),

            nn.AvgPool2d(3, 2)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x


In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
              ReLU-2            [-1, 4, 28, 28]               0
       BatchNorm2d-3            [-1, 4, 28, 28]               8
            Conv2d-4            [-1, 4, 28, 28]             148
              ReLU-5            [-1, 4, 28, 28]               0
       BatchNorm2d-6            [-1, 4, 28, 28]               8
            Conv2d-7            [-1, 8, 28, 28]             296
              ReLU-8            [-1, 8, 28, 28]               0
       BatchNorm2d-9            [-1, 8, 28, 28]              16
          Dropout-10            [-1, 8, 28, 28]               0
        MaxPool2d-11            [-1, 8, 14, 14]               0
           Conv2d-12           [-1, 12, 14, 14]             876
             ReLU-13           [-1, 12, 14, 14]               0
      BatchNorm2d-14           [-1, 12,

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.Resize((28, 28)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# Create a validation/test set from the training data
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [50000, 10000])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

def test(model, device, test_loader,epoch,optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Epoch: {epoch} | Learning Rate: {optimizer.param_groups[0]['lr']:.5f} | Test Loss: {test_loss:.4f} | Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")
    return test_loss

In [12]:
model = Net().to(device)
optimizer= optim.Adam(model.parameters(), lr= 0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader,epoch,optimizer)
    scheduler.step(test_loss)


loss=0.017699357122182846 batch_id=390: 100%|██████████| 391/391 [00:57<00:00,  6.83it/s]


Epoch: 1 | Learning Rate: 0.01000 | Test Loss: 0.1066 | Accuracy: 9673/10000 (96.73%)


loss=0.010867567732930183 batch_id=390: 100%|██████████| 391/391 [00:52<00:00,  7.44it/s]


Epoch: 2 | Learning Rate: 0.01000 | Test Loss: 0.0608 | Accuracy: 9807/10000 (98.07%)


loss=0.11412211507558823 batch_id=390: 100%|██████████| 391/391 [00:52<00:00,  7.47it/s]


...

Epoch: 17 | Learning Rate: 0.00100 | Test Loss: 0.0206 | Accuracy: 9945/10000 (99.45%)


loss=0.0006417831173166633 batch_id=390: 100%|██████████| 391/391 [00:52<00:00,  7.49it/s]


Epoch: 18 | Learning Rate: 0.00100 | Test Loss: 0.0203 | Accuracy: 9944/10000 (99.44%)


loss=0.0014463502448052168 batch_id=390: 100%|██████████| 391/391 [00:52<00:00,  7.48it/s]


Epoch: 19 | Learning Rate: 0.00100 | Test Loss: 0.0200 | Accuracy: 9949/10000 (99.49%)
